In [1]:
import pandas as pd
import numpy as np
import os
from datetime import datetime, date, timedelta 

from matplotlib import pyplot as plt
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)


In [2]:
# Licencee name removed with error shown
# to run this code, require Stata 17 and above with Python integration
import stata_setup
stata_setup.config("C:/Program Files/Stata18/", "be")

import pystata
from pystata import stata
from sfi import Scalar, Matrix, Data

ModuleNotFoundError: No module named 'pystata'

In [3]:
def setDate(df, start, end):
    start = datetime.strptime(start+"-01","%Y-%m-%d")
    end = datetime.strptime(end+"-01","%Y-%m-%d")
    df["Yr_Mth1"] = df["Yr_Mth"].map(lambda x : datetime.strptime(x+"-01","%Y-%m-%d"))
    df = df.loc[(df["Yr_Mth1"] <= end) & (df["Yr_Mth1"] >= start)]
    return df

In [4]:
def checkDuplicates(df):
    dfa1 = df.loc[:,["permid","Yr_Mth"]]
    dfa2 = dfa1.groupby(["permid","Yr_Mth"])["permid"].count()
    df_duplicates = dfa2.loc[:,dfa2 > 1]  # len 18
    return df_duplicates

In [3]:
pwd = "drive path"
#print(os.getcwd())

### Carbon Future prices and other instrumental variable returns

List of variables:
- MO : Symbol for EUA futures price
- CO : Symbol for Brent future price
- CBOT: Chicago Board of Trade ehtanol price
- CL : WTI price
- TTF : Natural gas price benchmark in Europe

In [6]:
df_carbon_3m = pd.read_csv("final data\\df_carbon_3m.csv")
df_carbon_3m.head()

,Yr_Mth,mo1_price,mo3_price,mo12_price,cl1_price,cl3_price,cl12_price,co1_price,co3_price,co12_price,cbot_eth_q1,cbot_eth_q4,ttf_m1,ttf_m3
0,2005-04,19.119000,19.118000,NaN,53.179184,55.005918,55.107959,52.750208,53.973125,53.536042,NaN,NaN,NaN,NaN
1,2005-07,23.135606,23.125000,NaN,63.312188,64.538281,64.682187,61.908769,63.050462,63.215385,NaN,NaN,NaN,NaN
2,2005-10,21.756923,21.757692,NaN,60.046774,60.934032,61.878387,57.760937,58.954687,60.379375,NaN,NaN,NaN,NaN
3,2006-01,25.554615,25.585385,NaN,63.476774,65.360161,67.542419,62.711094,63.838594,65.942969,NaN,NaN,NaN,NaN
4,2006-04,18.560000,18.973846,20.3,70.722698,72.565556,73.997302,70.361587,71.591905,72.649683,NaN,NaN,NaN,NaN


In [7]:
df_carbon_3m_ret = pd.read_csv("final data\\df_carbon_3m_ret.csv")
df_carbon_3m_ret.loc[df_carbon_3m["mo1_price"] > 0.8, "mo1_price"]  = 0.8  
df_carbon_3m_ret.loc[df_carbon_3m["mo1_price"] < -0.8, "mo1_price"] = -0.8
df_carbon_3m_ret = df_carbon_3m_ret.rename(columns = {"mo1_price": "mo1_ret", "mo3_price": "mo3_ret", \
    "mo12_price":"mo12_ret", "cl1_price":"cl1_ret", "cl3_price": "cl3_ret","cl12_price": "cl12_ret", \
    "co1_price":"co1_ret", "co3_price":"co3_ret","co12_price":"co12_ret", \
    "cbot_eth_q1":"cbot_eth_q1_ret","cbot_eth_q4":"cbot_eth_q4_ret","ttf_m1":"ttf_m1_ret","ttf_m3":"ttf_m3_ret"})
df_carbon_3m_ret.head()

,Yr_Mth,mo1_ret,mo3_ret,mo12_ret,cl1_ret,cl3_ret,cl12_ret,co1_ret,co3_ret,co12_ret,cbot_eth_q1_ret,cbot_eth_q4_ret,ttf_m1_ret,ttf_m3_ret
0,2005-04,0.8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2005-07,0.8,0.209593,NaN,0.190545,0.173297,0.173736,0.173621,0.168183,0.180800,NaN,NaN,NaN,NaN
2,2005-10,0.8,-0.059127,NaN,-0.051576,-0.055847,-0.043347,-0.066999,-0.064960,-0.044863,NaN,NaN,NaN,NaN
3,2006-01,0.8,0.175924,NaN,0.057122,0.072638,0.091535,0.085701,0.082842,0.092144,NaN,NaN,NaN,NaN
4,2006-04,0.8,-0.258411,NaN,0.114151,0.110241,0.095568,0.121996,0.121452,0.101705,NaN,NaN,NaN,NaN


## Read in stock data
For both US and Germany
- Returns
- Fundamental data
    - Momentum
    - Book to mkt value ratio
    - Mkt capitalisation
    - Profit indicators: REQ (for EU) retained earnings &  (for US)
- ESG sentiment data: Emissions and EnvironmentalInnovation
- Class: brown, green or consumer

In [8]:
df_gr = pd.read_csv("final data\\final_data_gr.csv")
df_gr = setDate(df_gr, "2006-01", "2022-02")
df_gr.head()

,Unnamed: 0,Yr_Mth,permid,MomentumL1Y,Mth_Ret,mkt_cap,req,bm,Emissions,EnvironmentalInnovation,buzz,class,Yr_Mth1
18,18,2006-01,4295868725,0.066941,0.101909,22.850595,NaN,123.235305,0.151157,7.218342,1117.283333,consumer,2006-01-01
19,19,2006-04,4295868725,0.091841,-0.245961,23.288475,NaN,114.734866,-0.212662,1.235282,1448.236264,consumer,2006-04-01
20,20,2006-07,4295868725,-0.000135,-0.705511,22.818316,NaN,126.740292,0.339104,0.411439,1281.010870,consumer,2006-07-01
21,21,2006-10,4295868725,-0.204360,0.034518,22.852252,NaN,124.101557,-0.047341,0.212121,1145.831522,consumer,2006-10-01
22,22,2007-01,4295868725,-0.203761,-0.006564,22.790050,2324.0,129.003667,-0.421738,0.037840,1103.544444,consumer,2007-01-01


In [9]:
df_esg_3m = pd.read_csv("final data\\df_esg_3m.csv")
df_esg_3m = df_esg_3m[["permid","Yr_Mth","Emissions"]]

df_esg_3m = df_esg_3m.dropna()
df_esg_3m.head()

,permid,Yr_Mth,Emissions
0,4295270870,2000-01,66.400000
1,4295270870,2000-04,62.252747
2,4295270870,2000-07,51.108696
3,4295270870,2000-10,39.630435
4,4295270870,2001-01,35.952941


In [10]:
df_carbon_3m_1 = df_carbon_3m[["Yr_Mth", "mo1_price", "mo3_price", "co1_price", "co12_price", "cbot_eth_q1", "ttf_m1", "ttf_m3"]]
df_gr_f = pd.merge(df_gr, df_carbon_3m_1, left_on="Yr_Mth", right_on = "Yr_Mth")

df_carbon_3m_2 = df_carbon_3m_ret[["Yr_Mth", "mo1_ret", "mo3_ret", "co1_ret", "co12_ret", "cbot_eth_q1_ret", "ttf_m1_ret", "ttf_m3_ret"]]
df_gr_f = pd.merge(df_gr_f, df_carbon_3m_2, left_on="Yr_Mth", right_on = "Yr_Mth")

df_gr_f = df_gr_f.drop("EnvironmentalInnovation", axis=1)
df_gr_f = df_gr_f.rename(columns = {"Emissions":"Emissions_ret" })

df_gr_f = pd.merge(df_gr_f, df_esg_3m, left_on=["permid","Yr_Mth"], right_on = ["permid","Yr_Mth"])
df_gr_f.head()
# Retained Earnings -> req

,Unnamed: 0,Yr_Mth,permid,MomentumL1Y,Mth_Ret,mkt_cap,req,bm,Emissions_ret,buzz,...,ttf_m1,ttf_m3,mo1_ret,mo3_ret,co1_ret,co12_ret,cbot_eth_q1_ret,ttf_m1_ret,ttf_m3_ret,Emissions
0,18,2006-01,4295868725,0.066941,0.101909,22.850595,NaN,123.235305,0.151157,1117.283333,...,NaN,NaN,0.8,0.175924,0.085701,0.092144,NaN,NaN,NaN,63.588889
1,110,2006-01,4295869176,-0.070802,0.138177,19.176340,93.995,9.364560,0.715761,995.777778,...,NaN,NaN,0.8,0.175924,0.085701,0.092144,NaN,NaN,NaN,54.811111
2,187,2006-01,4295869236,0.120675,0.191996,23.215718,2271.600,173.046554,0.169671,678.200000,...,NaN,NaN,0.8,0.175924,0.085701,0.092144,NaN,NaN,NaN,65.488889
3,262,2006-01,5040706960,0.045522,0.007279,22.020845,1108.192,61.866745,0.192376,632.650000,...,NaN,NaN,0.8,0.175924,0.085701,0.092144,NaN,NaN,NaN,20.400000
4,337,2006-01,4295868692,0.007028,0.098667,23.348876,NaN,226.503408,-0.196032,134.300000,...,NaN,NaN,0.8,0.175924,0.085701,0.092144,NaN,NaN,NaN,44.244444


### GMM for Germany 
Fixed effects: industry (green, brown and consumer) + Year (xtset)
(Firm)
Dependent variable: stock returns (monthly for US & quarterly for Germany)
Entity-specific regressor: Emissions, Momentum, Book to market, Mkt cap (heterogenous)
Common regressor: Carbon returns

there is a new Stata command xtreg -vxtdpdgmm
Reference for GMM in https://www.stata.com/manuals13/rgmm.pdf
https://www.statalist.org/forums/forum/general-stata-discussion/general/1567553-s-gmm-interpretation
https://www.statalist.org/forums/forum/general-stata-discussion/general/1395858-xtdpdgmm-new-stata-command-for-efficient-gmm-estimation-of-linear-dynamic-panel-models-with-nonlinear-moment-conditions


            
           

In [11]:
# consider adding investment as one of the moment conditions
# req: retained earnings
list_dict = []
var = ["beta_1","cons_1","beta_2","beta_3","cons_3"]
var_p = ["beta_1_p","cons_1_p","beta_2_p","beta_3_p","cons_3_p"]
for industry in ["brown"]: #,"green","consumer"]:
    dfa1 = df_gr_f[df_gr_f["class"]==industry]  # depends a lot on industry
    stata.pdataframe_to_data(dfa1, force=True)
    try:
        stata.run('''gen Yr_Mth2 = date(Yr_Mth, "YM")''')
        df_dates = pystata.stata.pdataframe_from_data(["Yr_Mth2", "Yr_Mth"])

        #stata.run('''asreg Mth_Ret bm MomentumL1Y mkt_cap Emissions, fmb newey(2) first save(results) se''')
        stata.run('''xtset permid Yr_Mth2''')
        stata.run('''gen lag_mo =mo1_ret[_n-1]''')
        stata.run('''gen lag_emt=Emissions_ret[_n-2]''')
        stata.run('''gen lag_em =(0.01*Emissions[_n-5]-0.5)''')
        stata.run('''gen lag_cbot=cbot_eth_q1[_n-2]''')
        stata.run('''gen lag_ttf =ttf_m3_ret[_n-2]''')
        stata.run('''gen lag_brent =co12_ret[_n-2]''')  # Brent
        #stata.run('''ds''') #  *lag_em with and without to test robustness ; display variables in memory
        stata.run('''gmm (eq1: Mth_Ret-{beta_1}*mo1_ret*lag_em-{cons_1}) (eq2: Emissions_ret-{beta_2}*lag_emt-0.005) \
            (eq3: req-{beta_3}*max(mo1_price-20,0)-{cons_3}), \
            instruments(eq1 eq3: lag_ttf lag_brent lag_em) winitial(identity)''')
        
        #stata.run(''' reg req mo1_ret''')  #mo1_ret

        df_vals = pystata.stata.get_return()
        df_vale = pystata.stata.get_ereturn() 
        res  = dict(zip(var, df_vals["r(PT)"][:,0]))
        df_p = dict(zip(var_p, df_vals["r(PT)"][:,3]))
        res.update(df_p)

        res["class"] = industry 
        res["no_of_parameters"] = 5
        res["no_of_moments"] = 9
        res["no_of_obs"] = df_vale["e(N)"]
        
        stata.run('''estat overid''')
        df_valsHS = pystata.stata.get_return()
        res["Hansen_prob"] = df_valsHS["r(J_p)"]
        res["Hansen_J_stat"] = df_valsHS["r(J)"]
        list_dict.append(res)
    except Exception as e: 
        print ("ERROR: " + str(e))
df_res = pd.DataFrame.from_records(list_dict, index = range(len(list_dict)))


Panel variable: permid (unbalanced)
 Time variable: Yr_Mth2, 16802 to 22646, but with gaps
         Delta: 1 unit
(1 missing value generated)
(58 missing values generated)
(5 missing values generated)
(1,187 missing values generated)
(172 missing values generated)
(2 missing values generated)
note: 106 missing values returned for equation 2 at initial values.
note: 360 missing values returned for equation 3 at initial values.

Step 1
Iteration 0:  GMM criterion Q(b) =    3992754  
Iteration 1:  GMM criterion Q(b) =  126294.45  
Iteration 2:  GMM criterion Q(b) =  126294.45  

Step 2
Iteration 0:  GMM criterion Q(b) =  .16087326  
Iteration 1:  GMM criterion Q(b) =  .05183609  
Iteration 2:  GMM criterion Q(b) =  .05183609  (backed up)

GMM estimation 

Number of parameters =   5
Number of moments    =   9
Initial weight matrix: Identity                   Number of obs   =      2,723
GMM weight matrix:     Robust

------------------------------------------------------------------------

In [12]:
df_vals

{'r(PT_has_legend)': 0.0,
 'r(PT_has_cnotes)': 0.0,
 'r(PT_k_ctitles)': 2.0,
 'r(level)': 95.0,
 'r(PT_rseps)': '`""\' `""\' `""\' `""\' `""\'',
 'r(PT_rnotes)': '`""\' `""\' `""\' `""\' `""\'',
 'r(PT_raligns)': '`"right"\' `"right"\' `"right"\' `"right"\' `"right"\'',
 'r(PT_rtitles)': '`"/beta_1"\' `"/cons_1"\' `"/beta_2"\' `"/beta_3"\' `"/cons_3"\'',
 'r(PT_cformats)': '`"%9.0g"\' `"%9.0g"\' `"%8.2f"\' `"%5.3f"\' `"%9.0g"\' `"%9.0g"\'',
 'r(PT_cspans2)': '`"1"\' `"1"\' `"1"\' `"1"\' `"2"\' `"0"\'',
 'r(PT_ctitles2)': '`"Coefficient"\' `"std. err."\' `"z"\' `"P>|z|"\' `"[95% conf. interval]"\' `""\'',
 'r(PT_cspans1)': '`"1"\' `"1"\' `"1"\' `"1"\' `"1"\' `"1"\'',
 'r(PT_ctitles1)': '`""\' `"Robust"\' `""\' `""\' `""\' `""\'',
 'r(put_tables)': 'PT',
 'r(citype)': 'normal',
 'r(_collect_prefix_get)': 'ignore',
 'r(PT)': array([[2.02191592e-01, 3.78443188e-02, 5.34271982e+00, 9.15622327e-08,
         1.28018090e-01, 2.76365094e-01],
        [2.38472980e-02, 7.98197005e-03, 2.98764564e

In [13]:
df_vale

{'e(rank)': 5.0,
 'e(N)': 2723.0,
 'e(Q)': 0.051836094651626814,
 'e(J)': 141.1496857363798,
 'e(J_df)': 4.0,
 'e(k_1)': 2.0,
 'e(k_2)': 1.0,
 'e(k_3)': 2.0,
 'e(converged)': 1.0,
 'e(has_xtinst)': 0.0,
 'e(type)': 1.0,
 'e(version)': 18.0,
 'e(Q_criterion)': 0.051836094651626814,
 'e(n_eq)': 3.0,
 'e(k)': 5.0,
 'e(n_moments)': 9.0,
 'e(k_eq)': 5.0,
 'e(k_aux)': 5.0,
 'e(k_eq_model)': 0.0,
 'e(scorevers)': 'version 18:',
 'e(cmdline)': 'gmm (eq1: Mth_Ret-{beta_1}*mo1_ret*lag_em-{cons_1}) (eq2: Emissions_ret-{beta_2}*lag_emt-0.005) (eq3: req-{beta_3}*max(mo1_price-20,0)-{cons_3}), instruments(eq1 eq3: lag_ttf lag_brent lag_em) winitial(identity)',
 'e(cmd)': 'gmm',
 'e(estat_cmd)': 'gmm_estat',
 'e(predict)': 'gmm_p',
 'e(marginsnotok)': 'Residuals SCores',
 'e(marginsok)': 'xb',
 'e(marginsprop)': 'allcons nochainrule',
 'e(eqnames)': 'eq1 eq2 eq3',
 'e(technique)': 'gn',
 'e(group)': 'permid',
 'e(winit)': 'Identity',
 'e(estimator)': 'twostep',
 'e(wmatrix)': 'robust',
 'e(vce)': 'ro

In [16]:
Wt = df_vale['e(S)']
df_wt = pd.DataFrame(Wt)
df_wt.to_clipboard()

In [18]:
conv = df_vale['e(converged)']
conv

1.0

In [12]:
index_list = ["beta_1","beta_1_p","cons_1","cons_1_p","beta_2","beta_2_p","beta_3","beta_3_p","cons_3","cons_3_p"]
param_list = ["no_of_obs","no_of_moments","no_of_parameters","Hansen_J_stat","Hansen_prob","class"]
df_res.loc[:,index_list] = df_res.loc[:,index_list].apply(lambda x : round(x,3))
df_res.head(5)
index_list.extend(param_list)

In [13]:
df_res_t = df_res.transpose()
df_res_t = df_res_t.reindex(index_list)
df_res_t.to_clipboard()
df_res_t.head(20)

,0
beta_1,0.202
beta_1_p,0.0
cons_1,0.024
cons_1_p,0.003
beta_2,1.075
beta_2_p,0.0
beta_3,34.448
beta_3_p,0.038
cons_3,830.397
cons_3_p,0.0


           #     instruments(1: lag_ttf lag_brent lag_em) \
           # instruments(3: lag_ttf lag_brent lag_em) \
             #   xtinstruments(1: Emissions, lags(2/.)) \
            #xtinstruments(3: Emissions, lags(2/.)) \
            # winitial(xt L L)''')
            #instruments(3 : lag_cbot lag_oil lag_e) winitial(identity)''')
            # instruments(2: lag_ttf ) \
        # a non-concave error is obtained if you do not have as many instruments as parameters fro estimation
        # Need to have more instruments than moments

### GMM for USA

In [12]:
#df_esg_1m = pd.read_csv("final data\\df_esg_1m.csv")
#df_esg_1m = df_esg_1m[["permid","Yr_Mth","Emissions"]]
#df_esg_1m = df_esg_1m.dropna()
#df_esg_1m.head()
# range from 1 to 100

In [ ]:
# winsorise returns that are way too high or too low
df_us_f.loc[df_us_f["Mth_Ret"] > 0.8, "Mth_Ret"]  = 0.8 # original without 518722 -->517675
df_us_f.loc[df_us_f["Mth_Ret"] < -0.8, "Mth_Ret"] = -0.8

df_us_f.loc[:,"mkt_cap"] = np.log10(df_us_f["mkt_cap"])
df_us_f.head()

In [11]:
#df_us = pd.read_csv("final data\\final_data_us.csv") 
#df_us = setDate(df_us, "2006-01", "2022-02")
#df_us = setDate(df_us, "2006-01", "2022-02")
#df_us.head()
# GProf - profitability data for US firms

In [4]:
# winsorise returns that are way too high or too low
#df_carbon_1m = pd.read_csv("final data\\df_carbon_1m.csv")
#df_carbon_1m.head()

,Yr_Mth,mo1_price,mo3_price,mo12_price,cl1_price,cl3_price,cl12_price,co1_price,co3_price,co12_price,cbot_eth_q1,cbot_eth_q4,ttf_m1,ttf_m3
0,2005-04,16.691667,16.691667,NaN,52.876667,55.073333,54.528333,53.228333,54.108333,52.863333,NaN,NaN,NaN,NaN
1,2005-05,17.625000,17.620455,NaN,49.870952,51.910476,52.085714,49.674000,50.878000,50.380500,NaN,NaN,NaN,NaN
2,2005-06,21.275000,21.277273,NaN,56.419545,57.942273,58.150909,55.416364,56.750000,56.588182,NaN,NaN,NaN,NaN
3,2005-07,24.619048,24.585714,NaN,59.026000,60.653000,60.936500,57.946667,59.369048,59.613333,NaN,NaN,NaN,NaN
4,2005-08,22.100000,22.100000,NaN,64.993478,66.263913,66.175217,63.800000,64.610455,64.580909,NaN,NaN,NaN,NaN


In [10]:
df_carbon_1m_1 = df_carbon_1m[["Yr_Mth", "mo1_price", "mo3_price", "co1_price", "co12_price", "cbot_eth_q1", "ttf_m1", "ttf_m3"]]
df_us_f = pd.merge(df_us, df_carbon_1m_1, left_on="Yr_Mth", right_on = "Yr_Mth")

df_carbon_1m_2 = df_carbon_1m_ret[["Yr_Mth", "mo1_ret", "mo3_ret", "co1_ret", "co12_ret", "cbot_eth_q1_ret", "ttf_m1_ret", "ttf_m3_ret"]]
df_us_f = pd.merge(df_us_f, df_carbon_1m_2, left_on="Yr_Mth", right_on = "Yr_Mth")
df_us_f.head()

df_us_f = df_us_f.drop("EnvironmentalInnovation", axis=1)
df_us_f = df_us_f.rename(columns = {"Emissions":"Emissions_ret" })

df_us_f = pd.merge(df_us_f, df_esg_1m, left_on=["permid","Yr_Mth"], right_on = ["permid","Yr_Mth"])
df_us_f.head()

NameError: name 'df_carbon_1m' is not defined

In [5]:
# winsorise returns that are way too high or too low
df_carbon_1m_ret = pd.read_csv("final data\\df_carbon_1m_ret.csv")
df_carbon_1m_ret.loc[df_carbon_1m["mo1_price"] > 0.8, "mo1_price"]  = 0.8  
df_carbon_1m_ret.loc[df_carbon_1m["mo1_price"] < -0.8, "mo1_price"] = -0.8
df_carbon_1m_ret = df_carbon_1m_ret.rename(columns = {"mo1_price": "mo1_ret", "mo3_price": "mo3_ret", \
    "mo12_price":"mo12_ret", "cl1_price":"cl1_ret", "cl3_price": "cl3_ret","cl12_price": "cl12_ret", \
    "co1_price":"co1_ret", "co3_price":"co3_ret","co12_price":"co12_ret", \
    "cbot_eth_q1":"cbot_eth_q1_ret","cbot_eth_q4":"cbot_eth_q4_ret","ttf_m1":"ttf_m1_ret","ttf_m3":"ttf_m3_ret"})
df_carbon_1m_ret.head()

,Yr_Mth,mo1_ret,mo3_ret,mo12_ret,cl1_ret,cl3_ret,cl12_ret,co1_ret,co3_ret,co12_ret,cbot_eth_q1_ret,cbot_eth_q4_ret,ttf_m1_ret,ttf_m3_ret
0,2005-04,0.8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2005-05,0.8,0.055644,NaN,-0.056844,-0.057430,-0.044795,-0.066775,-0.059701,-0.046967,NaN,NaN,NaN,NaN
2,2005-06,0.8,0.207533,NaN,0.131311,0.116196,0.116446,0.115601,0.115413,0.123216,NaN,NaN,NaN,NaN
3,2005-07,0.8,0.155492,NaN,0.046198,0.046783,0.047903,0.045660,0.046151,0.053459,NaN,NaN,NaN,NaN
4,2005-08,0.8,-0.101104,NaN,0.101099,0.092508,0.085970,0.101012,0.088285,0.083330,NaN,NaN,NaN,NaN


In [50]:
# consider adding investment as one of the moment conditions
list_dict = []
var = ["beta_1","cons_1","beta_2","beta_3","cons_3"]
var_p = ["beta_1_p","cons_1_p","beta_2_p","beta_3_p","cons_3_p"]
for industry in ["brown","green","consumer"]:
    dfa1 = df_us_f[df_us_f["class"]==industry]  # depends a lot on industry
    stata.pdataframe_to_data(dfa1, force=True)
    try:
        stata.run('''gen Yr_Mth2 = date(Yr_Mth, "YM")''')
        df_dates = pystata.stata.pdataframe_from_data(["Yr_Mth2", "Yr_Mth"])

        #stata.run('''asreg Mth_Ret bm MomentumL1Y mkt_cap Emissions, fmb newey(2) first save(results) se''')
        stata.run('''xtset permid Yr_Mth2''')
        stata.run('''gen lag_mo =mo1_ret[_n-1]''')
        stata.run('''gen lag_emt=Emissions_ret[_n-2]''')
        stata.run('''gen lag_em =(0.01*Emissions[_n-5]-0.5)''')
        stata.run('''gen lag_cbot=cbot_eth_q1[_n-2]''')
        stata.run('''gen lag_ttf =ttf_m3_ret[_n-2]''')
        stata.run('''gen lag_brent =co12_ret[_n-2]''')  # Brent
        #stata.run('''ds''') *lag_em
        stata.run('''gmm (eq1: Mth_Ret-{beta_1}*mo1_ret-{cons_1}) \
            (eq2: Emissions_ret-{beta_2}*lag_emt-0.005) \
            (eq3: GProf-{beta_3}*max(mo1_price-20,0)-{cons_3}), \
            instruments(eq1 eq3: lag_cbot lag_brent lag_em) winitial(identity)''')

        df_vals = pystata.stata.get_return()
        df_vale = pystata.stata.get_ereturn() 
        res  = dict(zip(var, df_vals["r(PT)"][:,0]))
        df_p = dict(zip(var_p, df_vals["r(PT)"][:,3]))
        res.update(df_p)

        res["class"] = industry 
        res["no_of_parameters"] = 5
        res["no_of_moments"] = 9
        res["no_of_obs"] = df_vale["e(N)"]
        
        stata.run('''estat overid''')
        df_valsHS = pystata.stata.get_return()
        res["Hansen_prob"] = df_valsHS["r(J_p)"]
        res["Hansen_J_stat"] = df_valsHS["r(J)"]
        list_dict.append(res)
    except Exception as e: 
        print ("ERROR: " + str(e))
df_res = pd.DataFrame.from_records(list_dict, index = range(len(list_dict)))


Panel variable: permid (unbalanced)
 Time variable: Yr_Mth2, 16802 to 22554, but with gaps
         Delta: 1 unit
(1 missing value generated)
(168 missing values generated)
(5 missing values generated)
(11,324 missing values generated)
(2,254 missing values generated)
(2 missing values generated)
note: 234 missing values returned for equation 2 at initial values.
note: 264 missing values returned for equation 3 at initial values.

Step 1
Iteration 0:  GMM criterion Q(b) =  .17478382  
Iteration 1:  GMM criterion Q(b) =  8.267e-06  
Iteration 2:  GMM criterion Q(b) =  8.267e-06  

Step 2
Iteration 0:  GMM criterion Q(b) =  .01240302  
Iteration 1:  GMM criterion Q(b) =  .01013827  
Iteration 2:  GMM criterion Q(b) =  .01013827  

GMM estimation 

Number of parameters =   5
Number of moments    =   9
Initial weight matrix: Identity                   Number of obs   =     16,103
GMM weight matrix:     Robust

------------------------------------------------------------------------------


In [53]:
index_list = ["beta_1","beta_1_p","cons_1","cons_1_p","beta_2","beta_2_p","beta_3","beta_3_p","cons_3","cons_3_p"]
param_list = ["no_of_obs","no_of_moments","no_of_parameters","Hansen_J_stat","Hansen_prob","class"]
df_res.loc[:,index_list] = df_res.loc[:,index_list].apply(lambda x : round(x,3))
df_res.head(5)
index_list.extend(param_list)

In [54]:
df_res_t = df_res.transpose()
df_res_t = df_res_t.reindex(index_list)
df_res_t.to_clipboard()
df_res_t.head(20)

,0,1,2
beta_1,0.397,1.354,1.489
beta_1_p,0.0,0.0,0.0
cons_1,-0.306,-1.069,-1.178
cons_1_p,0.0,0.0,0.0
beta_2,0.94,0.924,0.946
beta_2_p,0.0,0.0,0.0
beta_3,0.003,-0.003,0.002
beta_3_p,0.002,0.022,0.214
cons_3,0.201,0.215,0.418
cons_3_p,0.0,0.0,0.0
